In [1]:
import sys, os, distutils.core
import torch, detectron2
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [31]:
testIm = cv2.imread("images/personTest.jpg")
def crop(image, box):
  box = list(map(round, box))
  return image[box[1]:box[3],box[0]:box[2]]

In [10]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(testIm)

[02/12 09:32:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [11]:
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)
classes = outputs['instances'].pred_classes.to('cpu')
boxes = outputs['instances'].pred_boxes.to('cpu')
boxesNp = boxes.tensor.numpy()
masks = outputs['instances'].pred_masks.to('cpu')
print(masks[0].shape)
box1 = list(map(round, boxesNp[4]))

tensor([ 0, 63, 67, 41, 63, 56, 56, 56, 60, 62,  0, 56, 66, 62, 56],
       device='cuda:0')
Boxes(tensor([[176.8925, 128.9994, 442.2191, 558.4124],
        [318.5848, 328.7300, 608.0167, 527.5706],
        [281.6033, 469.0381, 389.1134, 538.0613],
        [712.3412, 255.5654, 736.3952, 292.0757],
        [640.4684, 359.5596, 750.0000, 559.3851],
        [545.6935, 302.9352, 724.0679, 421.3087],
        [606.0677, 237.1271, 642.6721, 281.5972],
        [629.0142, 223.1486, 664.2052, 281.1701],
        [569.3185, 279.2024, 750.0000, 315.0801],
        [ 53.7917,   1.7764, 387.1675, 227.8983],
        [433.2606, 260.1226, 545.5934, 407.7874],
        [ 48.0880, 336.0539, 206.6906, 563.0000],
        [353.1082, 432.7774, 454.2135, 511.1516],
        [554.8609, 142.3699, 571.1406, 205.0274],
        [643.4143, 220.1219, 660.9359, 270.4447]], device='cuda:0'))
torch.Size([563, 750])


In [ ]:
im= testIm

i = 0
im2 = crop(im, boxesNp[i])
#cv2.imshow('image',im2)
cv2.waitKey(0)
print(im.shape)

aMask = masks[i].unsqueeze(-1).numpy()
print(np.sum(aMask[:,:,0]))

im3 = crop(im*aMask,boxesNp[i])
#cv2.imshow("kasdfl",im3)
cv2.imwrite('testOut2.jpg', im3)

cv2.destroyAllWindows()

(563, 750, 3)
66908


1/1 [==============================] - 1s 506ms/step
--------
data and time:  2023-02-12 09:33:45.145276
colors:  {'beige': 4.850890491070459e-06, 'black': 3.575502205421799e-06, 'blue': 0.9992419481277466, 'brown': 1.352603362647642e-06, 'gold': 1.934447391249705e-05, 'green': 1.9015509678865783e-05, 'grey': 8.848131756167277e-07, 'orange': 0.0006274095503613353, 'pink': 1.7279497797062504e-06, 'purple': 7.076271799633105e-07, 'red': 2.8785383619833738e-05, 'silver': 2.2900569547346095e-06, 'tan': 8.387011689592327e-07, 'white': 2.4718242457311135e-06, 'yellow': 4.484656892600469e-05}
1/1 [==============================] - 0s 111ms/step
--------
data and time:  2023-02-12 09:33:53.649107
colors:  {'beige': 4.850890491070459e-06, 'black': 3.575502205421799e-06, 'blue': 0.9992419481277466, 'brown': 1.352603362647642e-06, 'gold': 1.934447391249705e-05, 'green': 1.9015509678865783e-05, 'grey': 8.848131756167277e-07, 'orange': 0.0006274095503613353, 'pink': 1.7279497797062504e-06, 'purple'

In [6]:
v = Visualizer(testIm[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow('Visualiser', out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
import gradio as gr
import tensorflow as tf
import numpy as np
from datetime import datetime


model = tf.keras.models.load_model("simple-CNN-model.2022-8-9.hdf5")

def image_predict(img):
    """
    Displays dominant colors beyond a given threshold.
    img : image input, for ex 'blue-car.jpg'
    isize: input image size, for ex. 227
    thr: chosen threshold value
    """
    thr=0
    global model
    if model is None:
        model = tf.keras.models.load_model("models/simple-CNN-model.2022-8-9.hdf5")
        
    data = np.asarray(img)
        
    ndata = np.expand_dims(data, axis=0)
    y_prob = model.predict(ndata/255)
    #y_prob.argmax(axis=-1)
    
    now = datetime.now()
    print("--------")
    print("data and time: ", now)
            
    colorlabels = ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']
    coltags = [sorted(colorlabels)[i] for i in np.where(np.ravel(y_prob)>thr)[0]]
    colprob = [np.ravel(y_prob)[i] for i in list(np.where(np.ravel(y_prob)>thr)[0])]
    
    if len(coltags) > 0:
        response = []
        for i,j in zip(coltags, colprob):
            #print(i,j)
            resp = {}
            resp[i] = float(j)
            response.append(resp)
        d = dict(map(dict.popitem, response))
        print('colors: ', d)
               
        return dict(d)

    else:
        return str('No label was found')

iface = gr.Interface(
    title = "Object color tagging",
    description = "App classifying objects on different colors",
    article = "<p style='text-align: center'><a href='https://www.rrighart.com' target='_blank'>Webpage</a></p>",
    fn=image_predict,
    inputs=gr.Image(shape=(227,227)), 
    outputs=gr.Label(),
    enable_queue=True,
    interpretation="default",
    debug=True
    )
#iface.launch()

c:\Users\Jean-Luc\Documents\PolyMtl\Git\.default\lib\site-packages\gradio\deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)
c:\Users\Jean-Luc\Documents\PolyMtl\Git\.default\lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'debug': True}
  warnings.warn(


In [26]:
def cropSquare(im):
    dim = min(im.shape[0], im.shape[1])
    if im.shape[0] > im.shape[1]:
        left = round((im.shape[0]-dim)//2)
        return im[left:left+dim]
    left = round((im.shape[1]-dim)//2)
    return im[:,left:left+dim]

resizeim3 = cv2.resize(cropSquare(im3), (227,227))
resizeim3 = cv2.cvtColor(resizeim3, cv2.COLOR_BGR2RGB)
#cv2.imshow('Visualiser', resizeim3)
cv2.waitKey(0)
cv2.destroyAllWindows()

def getBest(dct):
    best = None
    val = 0
    for item in dct:
        if dct[item] > val:
            best = item
            val = dct[item]
    return best, val


ans = getBest(image_predict(resizeim3))
print(ans)

1/1 [==============================] - 0s 97ms/step
--------
data and time:  2023-02-12 10:08:31.027726
colors:  {'beige': 4.560923116514459e-06, 'black': 3.6461119634623174e-06, 'blue': 0.9992368221282959, 'brown': 1.3800082570014638e-06, 'gold': 1.9600100131356157e-05, 'green': 2.0290563043090515e-05, 'grey': 8.125386443680327e-07, 'orange': 0.0006298327934928238, 'pink': 1.7705516484056716e-06, 'purple': 6.388135034285369e-07, 'red': 3.0910778150428087e-05, 'silver': 2.0637364741560305e-06, 'tan': 7.819139682396781e-07, 'white': 2.6793827601068188e-06, 'yellow': 4.4230586354387924e-05}
('blue', 0.9992368221282959)


In [34]:
def getItem(image):
    outputs = predictor(image)
    classes = outputs['instances'].pred_classes.to('cpu')
    boxes = outputs['instances'].pred_boxes.to('cpu')
    boxesNp = boxes.tensor.numpy()
    masks = outputs['instances'].pred_masks.to('cpu')

    done = False
    i = 0
    while not done and i < 10:
        aMask = masks[i].unsqueeze(-1).numpy()
        im3 = crop(image*aMask,boxesNp[i])
        resizeim3 = cv2.resize(cropSquare(im3), (227,227))
        resizeim3 = cv2.cvtColor(resizeim3, cv2.COLOR_BGR2RGB)
        color, val = getColor(resizeim3)
        if val > 0.8:
            done = True
        i += 1

    if not done:
        return (None, None, None)
    return color, classes[i], im3

def getColor(image):
    pred = image_predict(image)
    return getBest(pred)

In [36]:
color, cls, image = getItem(testIm)

cv2.imwrite('output.jpg', image)

print(color)

1/1 [==============================] - 0s 114ms/step
--------
data and time:  2023-02-12 10:13:43.749668
colors:  {'beige': 4.560923116514459e-06, 'black': 3.6461119634623174e-06, 'blue': 0.9992368221282959, 'brown': 1.3800082570014638e-06, 'gold': 1.9600100131356157e-05, 'green': 2.0290563043090515e-05, 'grey': 8.125386443680327e-07, 'orange': 0.0006298327934928238, 'pink': 1.7705516484056716e-06, 'purple': 6.388135034285369e-07, 'red': 3.0910778150428087e-05, 'silver': 2.0637364741560305e-06, 'tan': 7.819139682396781e-07, 'white': 2.6793827601068188e-06, 'yellow': 4.4230586354387924e-05}
blue
